In [7]:
import pandas as pd 
dataset = pd.DataFrame(pd.read_csv("Market_Basket_Analysis_v1.csv"))
dataset

,Order Date,Order ID,Customer ID,Ship Mode,Description,Information,Customer Name,Segment,City,State,Country,Market,Region,Attribute,Item Name,Quantity,Product ID,Unit Price,Total Sales
0,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name1,shrimp,167,10002439.0,74.69,12473.23
1,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name2,almonds,278,10000353.0,15.28,4247.84
2,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name3,avocado,324,10000994.0,46.33,15010.92
3,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name4,vegetables mix,124,10000475.0,58.22,7219.28
4,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name5,green grapes,680,10003338.0,86.31,58690.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72624,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name1,chicken,290,10004883.0,86.72,25148.80
72625,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name2,tropical fruit,314,10001990.0,56.47,17731.58
72626,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name3,other vegetables,447,10000425.0,92.13,41182.11
72627,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name4,vinegar,701,10001410.0,89.69,62872.69


In [8]:
product_set=dataset.groupby("Order ID")["Item Name"].apply(set).to_dict()

In [9]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
List_order_id=list(filter_basket.keys())
List_product_id=[list(values) for values in filter_basket.values()]
dict1={"Order_ID":List_order_id,"Product_ID":List_product_id}
df1=pd.DataFrame(dict1)
print(df1)

           Order_ID                                         Product_ID
0      AE-2013-1130  [frozen vegetables, escalope, tomato juice, eg...
1      AE-2013-1530  [cottage cheese, chocolate, turkey, cooking oi...
2      AE-2014-2840  [baking powder, fruit/vegetable juice, yogurt,...
3      AE-2014-4120                 [red/blush wine, other vegetables]
4      AG-2013-2840  [green tea, avocado, french fries, grated cheese]
...             ...                                                ...
12160  ZI-2014-6740  [tropical fruit, root vegetables, canned beer,...
12161  ZI-2014-6750                        [eggs, salad, french fries]
12162  ZI-2014-7160      [chicken, meat, other vegetables, whole milk]
12163  ZI-2014-9540  [whipped/sour cream, canned vegetables, whole ...
12164  ZI-2014-9550                            [soda, sugar, UHT-milk]

[12165 rows x 2 columns]


In [12]:
# Create a list of unique items
unique_items = sorted(set(item for sublist in df1['Product_ID'] for item in sublist))
 
# Initialize an empty dictionary to hold binary values for each item
binary_data = {item: [] for item in unique_items}
 
# Iterate over each row in the original DataFrame
for bought_together in df1['Product_ID']:
    # Initialize a binary row for the current transaction
    binary_row = {item: 0 for item in unique_items}
    # Set the value to 1 if the item was bought in the transaction
    for item in bought_together:
        binary_row[item] = 1
    # Append the binary row to the binary_data dictionary
    for item, value in binary_row.items():
        binary_data[item].append(value)
 
#Create a DataFrame from the dictionary
binary_df = pd.DataFrame(binary_data)
 
# Concatenate with the original DataFrame
binary_df2 = pd.concat([df1['Order_ID'], binary_df], axis=1)
 
# Display the binary DataFrame
print(binary_df)

       Instant food products  UHT-milk  abrasive cleaner  almonds  \
0                          0         0                 0        0   
1                          0         0                 0        0   
2                          0         0                 0        0   
3                          0         0                 0        0   
4                          0         0                 0        0   
...                      ...       ...               ...      ...   
12160                      0         0                 0        0   
12161                      0         0                 0        0   
12162                      0         0                 0        0   
12163                      0         0                 0        0   
12164                      0         1                 0        0   

       antioxydant juice  artif. sweetener  asparagus  avocado  babies food  \
0                      0                 0          0        0            0   
1            